# Najprostszy z możliwych model n-gramowy

In [1]:
import re
import pandas as pd
import funcs 
import importlib
import unicodecsv as csv
import pickle
import random
from timeit import default_timer as timer
from datetime import datetime, timedelta

importlib.reload(funcs)

<module 'funcs' from 'C:\\Users\\Mag\\Documents\\_projekty\\praca\\funcs.py'>

In [2]:
sp_joined = pd.read_csv('00_dane/sp_joined.csv',  sep='|', encoding='UTF-8')

In [5]:
%%time
n=1
words, words_model = list(), dict()
for speech in sp_joined['speech_raw']:
    words = list()
    words.extend(speech.split())
    for i in range(len(words)-n):
        speech_part = ' '.join(words[i:i+n])
        if speech_part not in words_model.keys():
            words_model[speech_part]=[]
        words_model[speech_part].append(words[i+n])
        
#Wall time: 3min 30s        

Wall time: 2min 14s


In [6]:
model1 = words_model

In [4]:
model2 = words_model

In [3]:
file = open('00_dane/model1', 'rb')

# dump information to that file
words_model = pickle.load(file)

# close the file
file.close()

In [30]:
def spin_doctor(speech_start, steps, words_model, n):
    if len(speech_start)==0:
        r = random.randrange(len(words_model.keys()))
        speech_start = list(words_model.keys())[r]
        
    if speech_start in words_model.keys():
        speech = list()
        words = speech_start
        speech.append(speech_start.capitalize())
        for _ in range(steps):
            possibilities = words_model[words]
            next_item = possibilities[random.randrange(len(possibilities))]
            speech.append(next_item)
            words = ' '.join(speech[-n:])
            words = ' '.join(words.split()[-n:])
            
        return ' '.join(speech)+'.'
    else:
        print('Brak w modelu')

Output modelu na 1-gram

In [18]:
print(spin_doctor('', 60, model1, 1))

Urobek podkomisji, mają stosować się zadania i tylko chęci ani upraw na czym operatorzy i przytułków dla polskiej żeglugi śródlądowej do Prawa Podatkowego, która zwalnia urzędników. Muszę zresztą widać w którym funkcjonuje już wyżej wymienionym projektem ustawy o których niesławnymi bohaterami narodowymi funduszami spójności terminologicznej. Wydaje się, że nasilają się zgodnie z mechanizmów, które toczą się teraz tego demo potrafi przeprowadzić.


In [22]:
print(spin_doctor('Dziś jest', 100, model2, 2))

Dziś jest już ustalony system finansów publicznych? Czy może mi pan minister mówi, że ta krytyka zasługuje na aprobatę argument zakładający, iż prezes Izby Cywilnej oraz Izby Kontroli w dniu 2 marca 2006 r. o świadczeniach przedemerytalnych też jest możliwe. Zdaniem prezydenta narusza to uprawnień przedsiębiorcy, naprawienie szkody wyrządzonej czynem niedozwolonym oraz pracowników cywilnych. Dla obydwu grup wyważyć. Pamiętajmy także o zwykłą większość głosów. Co to oznacza? Jest to bardzo ważna kwestia, to mianowicie, że w tej sytuacji pogłębia fakt, że pomoc państwa dla firm oraz poparcie dla tego rodzaju działalności tak specyficznych instytucji, jakimi są Policja, ośrodki pomocy społecznej regulują oczywiście.


In [35]:
print(spin_doctor('', 100, model2, 2))

Złudzeń, jakie są koszty badania i analizy odnoszące się do dynamiki eksportu nad importem alkoholi. Takim skutecznym i wystarczającym do uruchomienia Krajowego Centrum Hodowli Zwierząt na temat której dzisiaj mówimy o odsetkach wynoszących 12%, ale nie jest ustawą zasadniczą, którą postulowało rozwiązanie samorządowe. Czy ministerstwa podejmą działania na szkodę gospodarki, obywateli. Natomiast nie akceptujemy takich zachowań. Nawet jeśli jest ono słuszne. Aby się o swoje doświadczenia w zarządzaniu oświatą. W związku z tym, że księga wieczysta jest rejestrem centralnym, w budżecie na wydatki zwiazane z tą ostatnią, muszę powiedzieć, że to nie jest możliwe do zsumowania dopiero po 1 maja 2004 r..


## Spróbujmy ulepszyć model lepiej uwzględniając początek i koniec zdania.

In [3]:
dane_do_modelu = ' '.join(sp_joined['speech_raw'])

Usuńmy kropkę po "r.", przecinki, wielokrotne spacje.

In [4]:
dane_do_modelu = re.sub('\s+',' ',dane_do_modelu)
dane_do_modelu = re.sub('\ r.',' r',dane_do_modelu)

In [5]:
dane_do_modelu = re.sub('[!?.] ',' </s> <s> ',dane_do_modelu.lower())

In [6]:
dane_do_modelu = re.sub(',','',dane_do_modelu)

### Budowa modelu 1-gramowego

In [7]:
%%time
model1 = funcs.model_ng(dane_do_modelu, 3)

Wall time: 25min 49s


In [33]:
kl=list(model1.keys())

In [37]:
kl[100:120]

['w stanie',
 'stanie wojennym',
 'wojennym byli',
 'byli skazywani',
 'skazywani przez',
 'przez niekiedy',
 'niekiedy psychopatycznych',
 'psychopatycznych sędziów',
 'sędziów na',
 'na wyroki',
 'wyroki które',
 'które nie',
 'nie mają',
 'mają nic',
 'nic wspólnego',
 'wspólnego z',
 'z wymiarem',
 'wymiarem sprawiedliwości',
 'sprawiedliwości </s>',
 '<s> ale']

In [72]:
def spin_doctor(words_model, steps: int = 60, n: int =2, max_sent: int = 20):
    import random
    
    speech = list()
    first_words = [k.split()[1] for k in list(words_model.keys()) if k.split()[0]=='<s>']
    
    while len(speech)<steps:
        sentence=''
        
        w1 = first_words[random.randrange(len(first_words))]

        first_keys = [k for k in list(words_model.keys()) if k.split()[0]==w1]
        sent_start  = first_keys[random.randrange(len(first_keys))]

        words = sent_start
        sentence = sent_start.capitalize()
        sent_len=2
        ifend=0
        
        while sent_len<max_sent and ifend==0:
            possibilities = words_model[words]
            next_item = possibilities[random.randrange(len(possibilities))]
            
            if next_item == '</s>':
                ifend = 1
                speech.append(' '.join(sentence)+'.')
            else:
                sentence += ' ' + next_item
                words = ' '.join(sentence.split()[-n:])
                sent_len+=1
            print('Ifend = {}, sentence = {}, words = {}, next = {}'.format(ifend, sentence, words, next_item))
        
    return speech
    

In [73]:
spin_doctor(model1, 5, max_sent=20)

Ifend = 0, sentence = Fundamentalne rzbieżności co, words = rzbieżności co, next = co
Ifend = 0, sentence = Fundamentalne rzbieżności co do, words = co do, next = do
Ifend = 0, sentence = Fundamentalne rzbieżności co do złotówki, words = do złotówki, next = złotówki
Ifend = 0, sentence = Fundamentalne rzbieżności co do złotówki traktowały, words = złotówki traktowały, next = traktowały
Ifend = 0, sentence = Fundamentalne rzbieżności co do złotówki traktowały i, words = traktowały i, next = i
Ifend = 0, sentence = Fundamentalne rzbieżności co do złotówki traktowały i traktują, words = i traktują, next = traktują
Ifend = 0, sentence = Fundamentalne rzbieżności co do złotówki traktowały i traktują nasz, words = traktują nasz, next = nasz
Ifend = 0, sentence = Fundamentalne rzbieżności co do złotówki traktowały i traktują nasz kraj, words = nasz kraj, next = kraj
Ifend = 0, sentence = Fundamentalne rzbieżności co do złotówki traktowały i traktują nasz kraj europejskiej, words = kraj europe

['F u n d a m e n t a l n e   r z b i e ż n o ś c i   c o   d o   z ł o t ó w k i   t r a k t o w a ł y   i   t r a k t u j ą   n a s z   k r a j   e u r o p e j s k i e j   k a r c i e   s p o ł e c z n e j.',
 'B e z p o ś r e d n i e j   p r a c y   p r z y   m i n i s t r z e   s p o r t u.',
 'S z o p c e   r u c h   p a l i k o t a   w   s p r a w i e   u d z i e l a n i a   p o m o c y.',
 'S k o ń c z y ł   d a w n o.',
 'S p o r a   b i o r ą c   p o d   u w a g ę   o g ó l n e   s z a c u n k i   k t ó r e   b y ł y   p r o w a d z o n e   i n t e n s y w n i e   i   c h y b a   o r g a n i z a c y j n i e.']

In [56]:
t ='Złudzeń, jakie są koszty badania i analizy odnoszące się do dynamiki'

In [57]:
t[-2:]

'ki'